In [86]:
!ls

aita_analysis.Rproj  check_genderage.csv  reddit_data.ipynb
aita_analysis.qmd    extract-agegen.ipynb submissions.parquet


In [87]:
import pandas as pd

pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 10000)

In [88]:
df = pd.read_parquet("submissions.parquet")

In [89]:
df.head()

,link_flair_text,num_comments,over_18,score,url,selftext,title,id,edited,is_self,permalink,downs,ups,created
0,not the asshole,1,False,2,http://www.reddit.com/r/AmItheAsshole/comments/1fy0bx/aita_i_like_air_conditioning_and_my_coworkers/,"I work in an office that requires me to wear a suit all the time. I work more hours than everyone. Show up first, leave late, etc. The office has an air conditioning system which I turn on and set to 72º every morning. We don't run it at night. It's in New England and humid as fuck, so when you come in early, the place is dripping wet making it easy to feel uncomfortable at 70º when you always have to wear a coat and long pants. But after an hour of dehumidifying, 72º is fine. A co-worker of mine who only comes in once a week (and usually late) comes in dressed in designer clothes fit for going to the beach on a hot day and announces that it's too cold in here. I can't wear less and be dressed appropriately, and management isn't going to take the risk of criticizing a woman's fashion choices. I told her to wear a freaking sweater and left the AC on. *Am I the Asshole?*\n\n**TL;DR: I adjust the AC to make it comfortable in work-appropriate clothes. My co-worker wants the whole office hotter so she can dress like she's outside in the summer.**",AItA: I like air conditioning and my coworkers like working half-naked.,1fy0bx,0,True,/r/AmItheAsshole/comments/1fy0bx/aita_i_like_air_conditioning_and_my_coworkers/,0,2,NaN
1,too close to call,9,False,62,http://www.reddit.com/r/AmItheAsshole/comments/1ytr72/aita_construction_worker_here/,"I have been on a parking structure project for about 1 year and 3 months. People keep asking me when it will be completed and why it's taking so long. I am merely a foreman(assistant) on this job. I am very fed up with these questions and not completing it as well. I have started to vary my answer to 3 months plus a year for other new construction but that doesn't concern the opening as much. Or my other answers are laugh it off or get pissed, but don't give a direct answer. Am I an asshole for being so bitter over this project? \n\nIt's hard writing unbiased.",[AITA] Construction worker here,1ytr72,0,True,/r/AmItheAsshole/comments/1ytr72/aita_construction_worker_here/,0,62,1393275159
2,asshole,13,False,47,http://www.reddit.com/r/AmItheAsshole/comments/1ytxov/aita_i_wrote_an_explanation_in_til_and_came_off/,"[Here is the post in question](http://www.reddit.com/r/todayilearned/comments/1y40m9/til_a_circle_of_infinite_circumference_is_a/cfhhajs?context=3)\n\nIt seems like I came off as an asshole. What can I do in the future to avoid this? I thought I was just randomly explaining something, which seems common on Reddit.\n\nI definitely shouldn't have said ""Sorry for trying to help"" - That was pretty immature and rude. \n\nAm I the asshole here?\n\nI wrote as little as possible to skew perspective as little as possible.",[AITA] I wrote an explanation in TIL and came off as condescending,1ytxov,0,True,/r/AmItheAsshole/comments/1ytxov/aita_i_wrote_an_explanation_in_til_and_came_off/,0,47,1393278651
3,asshole,27,False,140,http://www.reddit.com/r/AmItheAsshole/comments/1yu29c/aita_threw_my_parents_donuts_away/,"My parents are diabetic, morbidly obese, and addicted to food. \n\nI'm the same, minus the diabetes. I've been trying for a while to eat healthily and lose weight. So have they, but yet every time they go to the store, they bring home junk food. \n\nI understand that it's hard for them, but it's also hard to stay on track with my healthier lifestyle when there's junk food all around me. Plus, they're getting older, so it is crucial for them to get healthy now.\n\n\nA few weeks ago, they brought home 4 bags of donuts, and I emptied each pack into the trash. Am I an asshole for this?\n\n**Edit:** I love all the responses. Got one ""no"", one ""maybe"", and one ""yes"". I agree with all of you.\n\n**Edit 2:** You guys are right--I'm the asshole.",[AITA] Threw my parent'

In [90]:
df.selftext

0                                                                                                                                                                                                                                                                                                                                                                   I work in an office that requires me to wear a suit all the time.  I work more hours than everyone.  Show up first, leave late, etc.  The office has an air conditioning system which I turn on and set to 72º every morning.  We don't run it at night.  It's in New England and humid as fuck, so when you come in early, the place is dripping wet making it easy to feel uncomfortable at 70º when you always have to wear a coat and long pants.  But after an hour of dehumidifying, 72º is fine.  A co-worker of mine who only comes in once a week (and usually late) comes in dressed in designer clothes fit for going to the beach on a hot day and annou

In [142]:
import re

# age & gender symbols (20F, 30M, 40F, etc.)
P = re.compile(r'\b(I|me|I\'m|im|am|my|myself)\s(\d{2}\s?(F|M)|(F|M)\s?\d{2})\b', re.IGNORECASE)

# age only
P_age = re.compile(r'\b(I|me|I\'m|im|am|my|myself)\s(\d{2})\b', re.IGNORECASE)

# gener only
P_gender_self = re.compile(r'\b(I|me|I\'m|im|am)\s(a\s)?(female|male|woman|man|mother|mom|dad|father)\b', re.IGNORECASE)
# P_gender_self = re.compile(r'\b(I|me|I\'m|im|am|my|myself)\s(a\s)?(female|male|woman|man|mother|mom|dad|father)\b', re.IGNORECASE)
P_gender_other = re.compile(r'\b(my)\s(ex-)?(wife|husband|girlfriend|boyfriend)\b', re.IGNORECASE)

gender_self = {
    "female": "F",
    "male": "M",
    "woman": "F",
    "man": "M",
    "mother": "F",
    "father": "M",
    "mom": "F",
    "dad": "M",
}

gender_other = {
    "wife": "M",
    "girlfriend": "M",
    "husband": "F",
    "boyfriend": "F",
}

def extract_age_gender(text):
    clean_text = text.lower()
    clean_text = re.sub(r"[()/,.\s]", " ", clean_text)
    clean_text = re.sub(r"\s{2,}", " ", clean_text)

    age = None
    gender = None

    age_gender_match = P.findall(clean_text)
    if age_gender_match:
        selfref = age_gender_match[0][0]
        symbol = age_gender_match[0][1]
        age = int("".join([char for char in symbol if char.isnumeric()]))
        gender = [char for char in symbol if char.isalpha()][0].upper()

    else:
        age_match = P_age.findall(clean_text)
        if age_match:
            age = int(age_match[0][1])

        gender_match = P_gender_self.findall(clean_text)
        if gender_match:
            gender = gender_self[gender_match[0][2]]

        gender_match = P_gender_other.findall(clean_text)
        if gender_match:
            gender = gender_other[gender_match[0][2]]

    return age, gender

In [143]:
df["age_gender"]=df.selftext.apply(lambda x: extract_age_gender(x))

In [144]:
df["age"] = df.age_gender.apply(lambda x:x[0])
df["gender"] = df.age_gender.apply(lambda x:x[1])

In [163]:
df.loc[~df.age.isna()].head(1)

,link_flair_text,num_comments,over_18,score,url,selftext,title,id,edited,is_self,permalink,downs,ups,created,age_gender,age,gender
10,asshole,42,False,33,http://www.reddit.com/r/AmItheAsshole/comments/1yuj95/aita_i_hacked_our_house_router_changed_the/,"Backstory: I'm a semi-professional competitive gamer (on a throwaway, won't go into much more detail) who goes to events, and plays for money. I live at home with my mother, stepfather, younger brother and younger sister.\n\nThe hours my team and I play are between 6pm and 12am every night, although not for 6 hours continuously; it may be 9pm till 12am, or 6pm till 10pm, etc. and obviously I'm talking during those hours. Whenever my mother tells me they're going to bed though, I always do my best to keep it down and get offline as soon as possible. Sometimes I'll be in an important game though, so it takes me a little while before I stop talking.\n\nOver the past few weeks or so, my console has started booting me offline without fail between the hours of 11pm and 12am. This is a huge issue because I'll be booted during a big game fairly often and I'll be unable to connect again until the next day. It also means it's a huge burden on my teammates as we often lose out on an hour of practice. It never really occurred to me that my stepfather could've put parental controls on (I'm 18).\n\nThen, last night, at the same time, my PC's internet went off too. It's usually just my console, so I was pretty confused. Then my browser came up with a window saying 'This system is being blocked from accessing the internet.', so I immediately knew what was up. I went on my phone, accessed the browser page from my phone and tried to disable it that way. I couldn't guess the password though, so I went to the 'password override' option which involves holding a button physically on the router, putting in the serial number, wireless key and a new password. I waited until everyone had gone to bed, then went downstairs and did that. After that I accessed the parental controls, and there was a block for both my Xbox and my PC from 23:00 to 07:00. I deleted both of them, went back upstairs and continued on my PC.\n\nSo now my stepfather doesn't know the new password (although he could do the same override I did if he figures out how) and I don't get booted offline anymore. Am I an asshole for doing this, all because I game competitively and am trying to win money from winning events?","[AITA] I 'hacked' our house router, changed the password and turned off all the parental controls",1yuj95,0,True,/r/AmItheAsshole/comments/1yuj95/aita_i_hacked_our_house_router_changed_the/,0,33,1393290715,"(18, None)",18,None


In [146]:
len(df)

2178385

In [147]:
len(df.loc[~df.age.isna()])

223469

In [165]:
223469/2178385

0.10258471298691461

In [149]:
len(df.loc[~df.gender.isna()])

296389

In [164]:
296389/2178385

0.13605905292223366

# Cleansed data

In [151]:
flair_list = [
 'Not the A-hole',
 'Asshole',
 'No A-holes here',
 'Everyone Sucks',
 'Not enough info',
 # 'UPDATE',
 # 'TL;DR',
 'not the a-hole',
 # 'POO Mode Activated 💩',
 'asshole',
 # '',
 # 'META',
 'Shitpost',
 'not the asshole',
 'no a--holes here',
 'everyone sucks',
 'too close to call',
 'not enough info']

In [152]:
df_c = df.loc[df.link_flair_text.isin(flair_list)]

In [153]:
len(df_c.loc[~df.age.isna()])/len(df_c)

0.27843756784597407

In [154]:
len(df_c.loc[~df.gender.isna()])/len(df_c)

0.37377530799185155

In [155]:
df_c.loc[~df.age.isna()][["selftext","age","gender","link_flair_text"]].head(100)

,selftext,age,gender,link_flair_text
10,"Backstory: I'm a semi-professional competitive gamer (on a throwaway, won't go into much more detail) who goes to events, and plays for money. I live at home with my mother, stepfather, younger brother and younger sister.\n\nThe hours my team and I play are between 6pm and 12am every night, although not for 6 hours continuously; it may be 9pm till 12am, or 6pm till 10pm, etc. and obviously I'm talking during those hours. Whenever my mother tells me they're going to bed though, I always do my best to keep it down and get offline as soon as possible. Sometimes I'll be in an important game though, so it takes me a little while before I stop talking.\n\nOver the past few weeks or so, my console has started booting me offline without fail between the hours of 11pm and 12am. This is a huge issue because I'll be booted during a big game fairly often and I'll be unable to connect again until the next day. It also means it's a huge burden on my teammates as we often lose out on an hour of practice. It never really occurred to me that my stepfather could've put parental controls on (I'm 18).\n\nThen, last night, at the same time, my PC's internet went off too. It's usually just my console, so I was pretty confused. Then my browser came up with a window saying 'This system is being blocked from accessing the internet.', so I immediately knew what was up. I went on my phone, accessed the browser page from my phone and tried to disable it that way. I couldn't guess the password though, so I went to the 'password override' option which involves holding a button physically on the router, putting in the serial number, wireless key and a new password. I waited until everyone had gone to bed, then went downstairs and did that. After that I accessed the parental controls, and there was a block for both my Xbox and my PC from 23:00 to 07:00. I deleted both of them, went back upstairs and continued on my PC.\n\nSo now my stepfather doesn't know the new password (although he could do the same override I did if he figures out how) and I don't get booted offline anymore. Am I an asshole for doing this, all because I game competitively and am trying to win money from winning events?",18,None,asshole
19,"I'm a college student. My first roommate dropped out in October, and I got a new one to start he 2nd semester. He's the worst. Here's some of the undesirable qualities about him.\n\n* Incredible snoring. I had no clue someone could snore this loud and not wake his or her self up until now. He sounds like a fucking animal sometimes. He tops it off by moaning in his sleep after the loudest snores. You can hear it throughout my entire suite, even with doors clothes. I haven't approached him about it, but I know he knows about it. A friend of his spent the night in our room a few weekends ago, and one of the first things she said was ""have you told him about your snoring??"" He kind of brushed it off and quickly changed the subject. So he knows. He has to know. It's too loud not to know. This is fine, you can't help snoring sometimes. That being said, he sleeps *12 fucking hours a day*. On Mondays like today he sleeps for 8 hours then naps for 4 hours during the day. Nothing like coming to your dorm from a long day of class to your jackhammer going berzerk in the bed next to you. I'm not to pleased. Why does he sleep so much? It connects to my next point...\n\n* He's a lazy fuck. He's very fat, and makes no attempt to change that. He has the attitude that everyone is perfect just the way they are, and no one should change. I think it's because it would be hard for him to change for the better. He eats insane amounts of food sometimes. In addition to this, he doesn't do anything recreational. All I see him do with his time is sleep, go to class (which is fantastic), lay on his bed with his laptop, and sit in the lounge of my dorm. He doesn't help with room keep up at all. Most of these qualities seem pretty innocuou

In [156]:
df_c.columns

Index(['link_flair_text', 'num_comments', 'over_18', 'score', 'url',
       'selftext', 'title', 'id', 'edited', 'is_self', 'permalink', 'downs',
       'ups', 'created', 'age_gender', 'age', 'gender'],
      dtype='object')

In [157]:
df_c.loc[:, df_c.columns !="age_gender"].to_parquet("check_genderage.parquet")

In [158]:
dt = pd.read_parquet("check_genderage.parquet")

In [159]:
dt.loc[~df.age.isna()][["selftext","age","gender","link_flair_text"]].head(100)

,selftext,age,gender,link_flair_text
10,"Backstory: I'm a semi-professional competitive gamer (on a throwaway, won't go into much more detail) who goes to events, and plays for money. I live at home with my mother, stepfather, younger brother and younger sister.\n\nThe hours my team and I play are between 6pm and 12am every night, although not for 6 hours continuously; it may be 9pm till 12am, or 6pm till 10pm, etc. and obviously I'm talking during those hours. Whenever my mother tells me they're going to bed though, I always do my best to keep it down and get offline as soon as possible. Sometimes I'll be in an important game though, so it takes me a little while before I stop talking.\n\nOver the past few weeks or so, my console has started booting me offline without fail between the hours of 11pm and 12am. This is a huge issue because I'll be booted during a big game fairly often and I'll be unable to connect again until the next day. It also means it's a huge burden on my teammates as we often lose out on an hour of practice. It never really occurred to me that my stepfather could've put parental controls on (I'm 18).\n\nThen, last night, at the same time, my PC's internet went off too. It's usually just my console, so I was pretty confused. Then my browser came up with a window saying 'This system is being blocked from accessing the internet.', so I immediately knew what was up. I went on my phone, accessed the browser page from my phone and tried to disable it that way. I couldn't guess the password though, so I went to the 'password override' option which involves holding a button physically on the router, putting in the serial number, wireless key and a new password. I waited until everyone had gone to bed, then went downstairs and did that. After that I accessed the parental controls, and there was a block for both my Xbox and my PC from 23:00 to 07:00. I deleted both of them, went back upstairs and continued on my PC.\n\nSo now my stepfather doesn't know the new password (although he could do the same override I did if he figures out how) and I don't get booted offline anymore. Am I an asshole for doing this, all because I game competitively and am trying to win money from winning events?",18,None,asshole
19,"I'm a college student. My first roommate dropped out in October, and I got a new one to start he 2nd semester. He's the worst. Here's some of the undesirable qualities about him.\n\n* Incredible snoring. I had no clue someone could snore this loud and not wake his or her self up until now. He sounds like a fucking animal sometimes. He tops it off by moaning in his sleep after the loudest snores. You can hear it throughout my entire suite, even with doors clothes. I haven't approached him about it, but I know he knows about it. A friend of his spent the night in our room a few weekends ago, and one of the first things she said was ""have you told him about your snoring??"" He kind of brushed it off and quickly changed the subject. So he knows. He has to know. It's too loud not to know. This is fine, you can't help snoring sometimes. That being said, he sleeps *12 fucking hours a day*. On Mondays like today he sleeps for 8 hours then naps for 4 hours during the day. Nothing like coming to your dorm from a long day of class to your jackhammer going berzerk in the bed next to you. I'm not to pleased. Why does he sleep so much? It connects to my next point...\n\n* He's a lazy fuck. He's very fat, and makes no attempt to change that. He has the attitude that everyone is perfect just the way they are, and no one should change. I think it's because it would be hard for him to change for the better. He eats insane amounts of food sometimes. In addition to this, he doesn't do anything recreational. All I see him do with his time is sleep, go to class (which is fantastic), lay on his bed with his laptop, and sit in the lounge of my dorm. He doesn't help with room keep up at all. Most of these qualities seem pretty innocuou

In [160]:
# df.head(100).style.set_table_styles([dict(selector='td', props='max-width: 600px;'),])

In [161]:
dt.loc[~df.gender.isna()][["selftext","age","gender","link_flair_text"]].head(100)

,selftext,age,gender,link_flair_text
9,"I love the musical *Les Miserables*. A coworker of mine said that they didn't like it. I asked why and they said, among other things, that ""it's not a musical, it's an opera.""\n\nNow, I happen to be an opera singer. I told them adamantly that it is NOT an opera. There are a number of things that make this true, but if you can take my word for it- and I hope you can- it's not a matter of opinion whether a mainstream musical is an opera or not. It is a musical, period.\n\nMy coworker thought otherwise. ""My boyfriend writes musicals and he agrees with me"" was the extent of her argument. Well, he may well write musicals but that doesn't mean he knows anything about opera. You might as well ask a jazz pianist to tell you about Beethoven- it's not the same area of expertise even if it's related. I told her ""You're entitled to your opinion even if it's wrong"" in a sort-of joking way. Maybe I was being dickish.\n\nI gotta know- am I the asshole?",NaN,F,asshole
14,"I work out of town so I'm usually home only 3 maybe 4 nights a week. Last week I spent Tuesday through Friday out of town and got home at about 8 pm Friday. Saturday my girlfriend had a meeting at 11 Am and it was supposed to be a couple hours, she went out with a coworker till about 11 pm when I picked her up and drove her home. I knew where she was, and I knew who she was with, I was a bit frustrated that I never got an invite, and I was annoyed that so much of our finite amount of time that we're home together was wasted. Instead of saying ""hey how bout you come home"" I just waiting thinking she'd show up at some point. We got in an argument about it then I went to bed. I feel like I am the asshole.",NaN,M,too close to call
21,"So my friend and I are seniors in high school and he starts to date this girl who is the worst. She treats him like shit and a bitch while he continues to tell us that he's in love and that they are perfect for each other. She is also a Debbie Downer and will never mind her own business which causes me and many of our mutual friends to dislike her a lot, however despite that fact I decided to be friendly towards her in order to keep my friend happy and her not so much of a problem. \n\nCut to a month or two ago when he is begging a bunch of our friends to let her come to a music festival with us because he promised her he would go with her without getting a place to stay. We do not like this idea at all but he just will not back down. He is so persistent about it that I finally snap and tell him that the reason she cannot come with is is because I hate her. Well this makes him riled up so he goes and tells his girlfriend the entire story. After that she would refuse to talk to me and wouldn't hang out with our group when I'm around, and wherever she goes he goes. \n\nCut to today where tells me it isn't fair she cannot hang out with our group, but my girlfriend is (she is very well liked by my friends.) We get to talking and he tells me to apologize to his girlfriend for making her feel bad because I ""pretended to be her friend,"" and I refused. Am I the asshole?",NaN,M,too close to call
25,"I don't get along with my in-laws very well. A lot of it comes down to us being from different cultures, but most of my well-intended mistakes have been interpreted as offensive. I do get along with my aunt &amp; uncle-in-law and they've attempted to stick up for me to my in-laws.\n\nMy cousin-in-law is getting married in a few weeks. I get along with her but I'm not close to her. I've been dreading the wedding because interactions with my in-laws are always very stressful.\n\nA month ago, my mom was diagnosed with extremely aggressive breast cancer. She's stage 4 with type 4 lymphoma. While the doctors have a lot of optimism, there's a very real chance it could be fatal. The doctors are rushing her into treatment before the cancer metastasizes. Her second chemo treatment, the ""bad one"" as the doctors call it, 